In [21]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [22]:
data_date = "2023-06-25"

In [23]:
df = pd.read_csv(data_date + "_data_cleaned.csv")

In [24]:
# Define the latitude and longitude for each county seat in the Czech Republic
county_seats = {
    "Praha": (50.0875, 14.421389),
    "České Budějovice": (48.974722, 14.474722),
    "Brno": (49.1951, 16.6068),
    "Karlovy Vary": (50.230556, 12.8725),
    "Jihlava": (49.400278, 15.590556),
    "Hradec Králové": (50.209167, 15.831944),
    "Liberec": (50.766667, 15.05),
    "Ostrava": (49.835556, 18.2925),
    "Olomouc": (49.593889, 17.250833),
    "Pardubice": (50.03861, 15.77916),
    "Plzeň": (49.741389, 13.3825),
    "Ústí nad Labem": (50.659167, 14.041667),
    "Zlín": (49.233056, 17.666944)
}

# Define central points for the north, west, south, and east parts of Prague
prague_points = {
    "Prague North": (50.126465, 14.412584),
    "Prague West": (50.061711, 14.296885),
    "Prague South": (50.006358, 14.416018),
    "Prague East": (50.081322, 14.541674)
}

# Combine the two dictionaries for all reference points
reference_points = {**county_seats, **prague_points}


In [25]:
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the Haversine distance between two points
    """
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # Radius of Earth in kilometers
    return c * r

# Calculate the distance from each listing to each reference point
for point_name, (lat, lon) in reference_points.items():
    df[f"Distance_to_{point_name.replace(' ', '_')}"] = haversine_distance(df["Latitude"], df["Longitude"], lat, lon)

# Display the first few rows of the augmented dataset
df.iloc[:, -17:].head()


,Distance_to_Praha,Distance_to_České_Budějovice,Distance_to_Brno,Distance_to_Karlovy_Vary,Distance_to_Jihlava,Distance_to_Hradec_Králové,Distance_to_Liberec,Distance_to_Ostrava,Distance_to_Olomouc,Distance_to_Pardubice,Distance_to_Plzeň,Distance_to_Ústí_nad_Labem,Distance_to_Zlín,Distance_to_Prague_North,Distance_to_Prague_West,Distance_to_Prague_South,Distance_to_Prague_East
0,5.125915,119.518673,181.355463,115.060600,108.586197,99.240024,89.965207,275.099993,206.360493,94.052773,84.585985,74.060298,247.957529,9.255142,11.876314,5.824673,6.685565
1,139.883166,200.001966,123.710915,247.927588,113.838462,38.712984,106.303305,147.183184,100.832512,50.264587,221.421798,168.881799,150.974262,139.795428,149.143597,142.145367,131.578243
2,89.255336,201.105451,197.895334,172.875383,149.906021,73.072459,10.923243,241.567748,193.230764,86.887718,168.660016,81.174320,242.571665,86.245070,97.062026,96.803872,84.906000
3,9.666503,120.360144,191.582837,103.557199,117.741602,110.714271,95.749310,286.781140,217.801147,105.744755,74.093284,70.026779,259.018430,11.688372,1.187412,9.762277,17.685913
4,4.418828,119.512668,182.807678,113.390294,109.854579,100.922152,90.789474,276.801161,208.015313,95.758849,83.011180,73.423015,249.548229,8.777582,10.187548,5.028597,8.197060


In [26]:
import geopandas as gpd
from shapely.geometry import Point


# Load the regions shapefile
regions = gpd.read_file('CZE_adm1.shp')

# If the regions GeoDataFrame has no CRS, set it to WGS 84 (EPSG:4326)
if regions.crs is None:
    regions = regions.set_crs("EPSG:4326")

# Assuming you have a DataFrame 'df' with 'Latitude' and 'Longitude'
df['Coordinates'] = df.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
points_gdf = gpd.GeoDataFrame(df, geometry='Coordinates')

# If the points GeoDataFrame has no CRS, set it to WGS 84 (EPSG:4326)
if points_gdf.crs is None:
    points_gdf = points_gdf.set_crs("EPSG:4326")

# Perform the spatial join
df = gpd.sjoin(points_gdf, regions, how='left', op='within')


In [27]:
# Load the CSV file into a DataFrame
regions_df = pd.read_csv('CZE_adm1.csv')

# Adjust the OBJECTID to be 0-indexed
regions_df['OBJECTID_0_indexed'] = regions_df['OBJECTID'] - 1

# Merge with your points DataFrame on index_right
# Select only the 'NAME_1' and 'OBJECTID_0_indexed' for merging
df = df.merge(regions_df[['NAME_1', 'OBJECTID_0_indexed']], left_on='index_right', right_on='OBJECTID_0_indexed', how='left')

# Drop the 'OBJECTID_0_indexed' as it's no longer needed after the merge
df.drop(['OBJECTID_0_indexed', 'Coordinates', 'index_right'], axis=1, inplace=True)

# Rename the 'NAME_1' column to 'Region'
df.rename(columns={'NAME_1': 'Region'}, inplace=True)

In [28]:
rename_pairs = {
    "Prague": "Praha",
    "Stredoceský": "Středočeský",
    "Jihoceský": "Jihočeský",
    "Kraj Vysocina": "Kraj Vysočina",
    "Plzenský": "Plzeňský"
}

# Rename the regions
df["Region"] = df["Region"].replace(rename_pairs)

In [29]:
df.Region.value_counts()

Region
Praha              3680
Středočeský        1860
Ústecký            1561
Jihomoravský       1436
Moravskoslezský    1196
Olomoucký           923
Karlovarský         920
Plzeňský            799
Jihočeský           771
Liberecký           679
Královéhradecký     575
Pardubický          458
Zlínský             442
Kraj Vysočina       393
Name: count, dtype: int64

In [30]:
city_region_pairs = {
    "Praha": "Praha",
    "České_Budějovice": "Jihočeský",
    "Brno": "Jihomoravský",
    "Karlovy_Vary": "Karlovarský",
    "Jihlava": "Kraj Vysočina",
    "Hradec_Králové": "Královéhradecký",
    "Liberec": "Liberecký",
    "Ostrava": "Moravskoslezský",
    "Olomouc": "Olomoucký",
    "Pardubice": "Pardubický",
    "Plzeň": "Plzeňský",
    "Ústí_nad_Labem": "Ústecký",
    "Zlín": "Zlínský",
}

def assign_region(row):
    # Filter out columns related to Prague directions we need to ignore
    distances = row.filter(regex='^Distance_to_(?!Prague_)')
    # Find the minimum distance
    min_distance = distances.min()
    # Find the city that corresponds to the minimum distance
    closest_city = distances.idxmin().split('Distance_to_')[1]
    # Return the corresponding region
    return city_region_pairs[closest_city]

# Apply the function across the dataframe row-wise only where "Region" is null
df.loc[df['Region'].isnull(), 'Region'] = df.loc[df['Region'].isnull()].apply(assign_region, axis=1)

# Output the dataframe with the assigned regions
print(df[['Region']])


                Region
0                Praha
1      Královéhradecký
2            Liberecký
3                Praha
4                Praha
...                ...
15724        Jihočeský
15725     Jihomoravský
15726  Královéhradecký
15727            Praha
15728  Moravskoslezský

[15729 rows x 1 columns]


In [31]:
# Provided salary data as a dictionary
salary_data = {
    "Region": ["Praha", "Středočeský", "Jihočeský", "Plzeňský", "Karlovarský", 
                    "Ústecký", "Liberecký", "Královéhradecký", "Pardubický", "Kraj Vysočina", "Jihomoravský",
                    "Olomoucký", "Zlínský", "Moravskoslezský"],
    "Průměrná hrubá měsíční mzda (Kč)": ["52 213 Kč", "43 730 Kč", "40 328 Kč", "41 520 Kč", 
                                         "38 019 Kč", "40 612 Kč", "39 752 Kč", "41 062 Kč", "39 345 Kč", 
                                         "40 291 Kč", "43 092 Kč", "39 594 Kč", "39 505 Kč", "39 563 Kč"],
    "Počet zaměstnanců (v tis.)": [869.7, 418, 219.4, 214.8, 82.4, 241, 144.1, 196.2, 182.7, 170.8, 
                                  467.4, 218.6, 203.4, 417.6]
}

# Convert the dictionary to a DataFrame
salary_df = pd.DataFrame(salary_data)

# Clean the salary column to remove "Kč" and convert to numeric
salary_df["Průměrná hrubá měsíční mzda (Kč)"] = salary_df["Průměrná hrubá měsíční mzda (Kč)"].str.replace(" Kč", "").str.replace(" ", "").astype(int)

# Display the cleaned salary dataframe
salary_df

,Region,Průměrná hrubá měsíční mzda (Kč),Počet zaměstnanců (v tis.)
0,Praha,52213,869.7
1,Středočeský,43730,418.0
2,Jihočeský,40328,219.4
3,Plzeňský,41520,214.8
4,Karlovarský,38019,82.4
5,Ústecký,40612,241.0
6,Liberecký,39752,144.1
7,Královéhradecký,41062,196.2
8,Pardubický,39345,182.7
9,Kraj Vysočina,40291,170.8


In [32]:
# Convert the salary data to a DataFrame for easier integration
salary_data_df = pd.DataFrame.from_dict(salary_data)

# Merge the property dataset with the salary data based on the region
merged_data = df.merge(salary_data_df, on='Region', how='left')

# Display the first few rows of the merged dataset to verify the integration
merged_data[['Latitude', 'Longitude', 'Region', 'Průměrná hrubá měsíční mzda (Kč)']].head()


,Latitude,Longitude,Region,Průměrná hrubá měsíční mzda (Kč)
0,50.049548,14.462156,Praha,52 213 Kč
1,50.296096,16.359198,Královéhradecký,41 062 Kč
2,50.723614,15.189542,Liberecký,39 752 Kč
3,50.051077,14.298411,Praha,52 213 Kč
4,50.049265,14.438260,Praha,52 213 Kč


In [33]:
# Convert the salary column to numeric format
merged_data['Průměrná hrubá měsíční mzda (Kč)'] = merged_data['Průměrná hrubá měsíční mzda (Kč)'].str.replace(" Kč", "").str.replace(" ", "").astype(int)

# Verify the data type and check the first few rows
merged_data['Průměrná hrubá měsíční mzda (Kč)'].dtype, merged_data[['Region', 'Průměrná hrubá měsíční mzda (Kč)']].head()


(dtype('int64'),
             Region  Průměrná hrubá měsíční mzda (Kč)
 0            Praha                             52213
 1  Královéhradecký                             41062
 2        Liberecký                             39752
 3            Praha                             52213
 4            Praha                             52213)

In [34]:
# Data for the number of people in each Region
population_data = {
    'Region': [
        'Praha', 'Středočeský', 'Jihočeský', 'Plzeňský', 
        'Karlovarský', 'Ústecký', 'Liberecký', 'Královéhradecký', 
        'Pardubický', 'Kraj Vysočina', 'Jihomoravský', 'Olomoucký', 
        'Zlínský', 'Moravskoslezský'
    ],
    'Population': [
        1301432, 1415463, 631803, 581436, 279103, 789098, 435220, 538303, 
        510037, 497661, 1197651, 619788, 564331, 1162841
    ]
}

# Convert to DataFrame
population_df = pd.DataFrame(population_data)

# Merge with the merged_data DataFrame
merged_data_with_population = merged_data.merge(population_df, on='Region', how='left')

# Display the first few rows to verify the merge
merged_data_with_population[['Region', 'Průměrná hrubá měsíční mzda (Kč)', 'Population']].head()


,Region,Průměrná hrubá měsíční mzda (Kč),Population
0,Praha,52213,1301432
1,Královéhradecký,41062,538303
2,Liberecký,39752,435220
3,Praha,52213,1301432
4,Praha,52213,1301432


In [35]:
# Crime data for each Region
crime_data = {
    'Region': [
        'Praha', 'Středočeský', 'Jihočeský', 'Plzeňský', 
        'Karlovarský', 'Ústecký', 'Liberecký', 'Královéhradecký', 
        'Pardubický', 'Kraj Vysočina', 'Jihomoravský', 'Olomoucký', 
        'Zlínský', 'Moravskoslezský'
    ],
    'Kriminalita celkem': [
        40704, 19775, 8916, 10207, 4939, 16365, 8399, 6659, 
        5230, 5240, 18579, 9114, 5980, 21884
    ],
    'Obecná kriminalita': [
        35056, 15703, 6511, 7889, 3648, 12512, 6551, 4955, 
        3797, 3950, 15152, 6566, 4211, 18026
    ],
    'Hospodářská kriminalita': [
        3040, 1049, 919, 866, 567, 1300, 687, 565, 
        339, 409, 1217, 904, 556, 1219
    ],
    'Loupeže': [
        255, 108, 57, 69, 47, 192, 67, 26, 
        34, 29, 166, 69, 42, 261
    ],
    'Vloupání do bytů a rodinných domů': [
        984, 873, 160, 228, 101, 356, 183, 142, 
        84, 91, 433, 92, 76, 470
    ],
    'Znásilnění': [
        153, 99, 35, 34, 26, 93, 56, 53, 
        33, 27, 98, 36, 28, 109
    ],
    'Vraždy': [
        24, 19, 3, 6, 10, 17, 2, 9, 
        10, 5, 10, 13, 7, 15
    ]
}

# Convert to DataFrame
crime_df = pd.DataFrame(crime_data)

# Merge with the enhanced_data DataFrame
enhanced_data_with_crime = merged_data_with_population.merge(crime_df, on='Region', how='left')

# Display the first few rows to verify the merge
enhanced_data_with_crime[['Region', 'Population', 'Kriminalita celkem', 'Obecná kriminalita']].head()

,Region,Population,Kriminalita celkem,Obecná kriminalita
0,Praha,1301432,40704,35056
1,Královéhradecký,538303,6659,4955
2,Liberecký,435220,8399,6551
3,Praha,1301432,40704,35056
4,Praha,1301432,40704,35056


In [36]:
# Create the property data
property_data = {
    'Region': [
        'Praha', 'Středočeský', 'Jihočeský', 'Plzeňský', 'Karlovarský',
        'Ústecký', 'Liberecký', 'Královéhradecký', 'Pardubický',
        'Kraj Vysočina', 'Jihomoravský', 'Olomoucký', 'Zlínský',
        'Moravskoslezský'
    ],
    'Rodinné domy (Kč za m2)': [
        96498, 57814, 39180, 38963, 32376, 32152, 39631, 37856, 35929, 35435, 43070, 32647, 31744, 34619
    ],
    'Byty (Kč za m2)': [
        100727, 46444, 41688, 42549, 29893, 30620, 39361, 41035, 42839, 39605, 56752, 39727, 39065, 35131
    ]
}

# Convert property data to DataFrame
property_df = pd.DataFrame(property_data)

# Merge property_df with enhanced_data_with_crime based on the 'Kraj' column
enhanced_data_with_property_prices = enhanced_data_with_crime.merge(property_df, on='Region', how='left')

# Display the first few rows to verify the added data
enhanced_data_with_property_prices[['Region', 'Rodinné domy (Kč za m2)', 'Byty (Kč za m2)']].head()

,Region,Rodinné domy (Kč za m2),Byty (Kč za m2)
0,Praha,96498,100727
1,Královéhradecký,37856,41035
2,Liberecký,39631,39361
3,Praha,96498,100727
4,Praha,96498,100727


In [37]:
# Calculate the theoretical price of the property
enhanced_data_with_property_prices['Teoretická cena'] = enhanced_data_with_property_prices['Užitná plocha'] * enhanced_data_with_property_prices['Byty (Kč za m2)']

# Display the first few rows with the new column
enhanced_data_with_property_prices[['Region', 'Celková cena', 'Užitná plocha', 'Byty (Kč za m2)', 'Teoretická cena']].head()

,Region,Celková cena,Užitná plocha,Byty (Kč za m2),Teoretická cena
0,Praha,6832000.0,48.0,100727,4834896.0
1,Královéhradecký,6361000.0,53.0,41035,2174855.0
2,Liberecký,2214000.0,35.0,39361,1377635.0
3,Praha,5643000.0,68.0,100727,6849436.0
4,Praha,5390000.0,38.0,100727,3827626.0


In [38]:
# Create a new DataFrame with foreigners data
foreigners_data = {
    'Region': [
        'Praha', 'Středočeský', 'Jihočeský', 'Plzeňský', 'Karlovarský',
        'Ústecký', 'Liberecký', 'Královéhradecký', 'Pardubický',
        'Kraj Vysočina', 'Jihomoravský', 'Olomoucký', 'Zlínský',
        'Moravskoslezský'
    ],
    'Total_foreigners': [
        345307, 158952, 47616, 80814, 38533,
        61437, 42622, 38806, 42564, 30805,
        110879, 28643, 26535, 56258
    ]
}

foreigners_df = pd.DataFrame(foreigners_data)

# Merge the two DataFrames on the 'Region' column
enhanced_data_with_foreigners = enhanced_data_with_property_prices.merge(foreigners_df, on='Region', how='left')

# Display the first few rows to verify the added data
enhanced_data_with_foreigners.head()


,url_id,Celková cena,Podlaží,Užitná plocha,Sklep,Parkování,Výtah,Latitude,Longitude,Terasa,...,Obecná kriminalita,Hospodářská kriminalita,Loupeže,Vloupání do bytů a rodinných domů,Znásilnění,Vraždy,Rodinné domy (Kč za m2),Byty (Kč za m2),Teoretická cena,Total_foreigners
0,722035788,6832000.0,2.0,48.0,1,1,1,50.049548,14.462156,0,...,35056,3040,255,984,153,24,96498,100727,4834896.0,345307
1,2418832972,6361000.0,1.0,53.0,0,1,0,50.296096,16.359198,0,...,4955,565,26,142,53,9,37856,41035,2174855.0,38806
2,3930260556,2214000.0,5.0,35.0,1,1,1,50.723614,15.189542,0,...,6551,687,67,183,56,2,39631,39361,1377635.0,42622
3,3756315724,5643000.0,4.0,68.0,1,0,1,50.051077,14.298411,0,...,35056,3040,255,984,153,24,96498,100727,6849436.0,345307
4,3955312204,5390000.0,4.0,38.0,1,0,1,50.049265,14.438260,0,...,35056,3040,255,984,153,24,96498,100727,3827626.0,345307


In [39]:
enhanced_data_with_foreigners = enhanced_data_with_foreigners.drop(enhanced_data_with_foreigners['Užitná plocha'].sort_values()[:50].index)

In [40]:
enhanced_data_with_foreigners.to_csv(data_date + "_data_cleaned_location_enhanced.csv", index=False)